# Laboratorio no calificado: Ajuste de hiperparámetros y entrenamiento de modelos con TFX

En este laboratorio, volverá a realizar el ajuste de hiperparámetros, pero esta vez, será dentro de un pipeline [Tensorflow Extended (TFX)](https://www.tensorflow.org/tfx/). 

Ya hemos introducido algunos componentes de TFX en el Curso 2 de esta especialización relacionados con la ingestión, validación y transformación de datos. En este cuaderno, llegarás a trabajar con dos más que están relacionados con el desarrollo y entrenamiento de modelos: *Tuner* y *Trainer*.

<img src='https://www.tensorflow.org/tfx/guide/images/prog_trainer.png' alt='tfx pipeline'>
image source: https://www.tensorflow.org/tfx/guide

* El *Tuner* utiliza la API [Keras Tuner](https://keras-team.github.io/keras-tuner/) para ajustar los hiperparámetros de su modelo.
* Puedes obtener el mejor conjunto de hiperparámetros del componente Tuner y alimentar el componente *Trainer* para optimizar tu modelo para el entrenamiento.

Volverá a trabajar con el conjunto de datos [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) y lo alimentará a través de la tubería TFX hasta el componente Trainer.Repasará rápidamente los componentes anteriores del Curso 2, y luego se centrará en los dos nuevos componentes introducidos.

Comencemos.



## Setup

### Instalar TFX

Primero instalará [TFX](https://www.tensorflow.org/tfx), un marco de trabajo para el desarrollo de pipelines de aprendizaje automático de extremo a extremo.

In [ ]:
!pip install --use-deprecated=legacy-resolver tfx==1.3.0
!pip install apache-beam==2.32.0

# These are downgraded to work with the packages used by TFX 1.3
# Please do not delete because it will cause import errors in the next cell
!pip install tensorflow==2.6.0
!pip install tensorflow-serving-api==2.6.0
!pip install --upgrade tensorflow-estimator==2.6.0
!pip install --upgrade keras==2.6.0

*Nota: En Google Colab, es necesario reiniciar el tiempo de ejecución en este punto para finalizar la actualización de los paquetes que acaba de instalar. Puede hacerlo haciendo clic en el botón "Reiniciar el tiempo de ejecución" al final de la celda de salida anterior (después de la instalación), o seleccionando "Tiempo de ejecución > Reiniciar el tiempo de ejecución" en la barra de menús. **Por favor, no pases a la siguiente sección sin reiniciar.** También puedes ignorar los errores de incompatibilidad de versiones de algunos de los paquetes incluidos porque no los usaremos en este cuaderno.*

### Imports

A continuación, importará los paquetes que necesitará para este ejercicio.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

import os
import pprint

from tfx.components import ImportExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
from tfx.components import Tuner
from tfx.components import Trainer

from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

## Descargar y preparar el conjunto de datos

Como se mencionó anteriormente, se utilizará el conjunto de datos Fashion MNIST al igual que en el laboratorio anterior. Esto le permitirá comparar las similitudes y diferencias al utilizar Keras Tuner como una biblioteca independiente y dentro de una tubería ML.

En primer lugar, tendrá que configurar los directorios que utilizará para almacenar el conjunto de datos, así como los artefactos de la tubería y el almacén de metadatos.

In [ ]:
# Location of the pipeline metadata store
_pipeline_root = './pipeline/'

# Directory of the raw data files
_data_root = './data/fmnist'

# Temporary directory
tempdir = './tempdir'

In [ ]:
# Create the dataset directory
!mkdir -p {_data_root}

# Create the TFX pipeline files directory
!mkdir {_pipeline_root}

Ahora descargará FashionMNIST desde [Tensorflow Datasets](https://www.tensorflow.org/datasets). La bandera `with_info` se establecerá en `True` para que pueda mostrar información sobre el conjunto de datos en la siguiente celda (es decir, utilizando `ds_info`).

In [ ]:
# Download the dataset
ds, ds_info = tfds.load('fashion_mnist', data_dir=tempdir, with_info=True)

In [ ]:
# Display info about the dataset
print(ds_info)

Puedes revisar los archivos descargados con el código de abajo. Para este laboratorio, utilizará el TFRecord *train* por lo que deberá tomar nota de su nombre de archivo. En este laboratorio no utilizarás el TFRecord *test*.

In [ ]:
# Define the location of the train tfrecord downloaded via TFDS
tfds_data_path = f'{tempdir}/{ds_info.name}/{ds_info.version}'

# Display contents of the TFDS data directory
os.listdir(tfds_data_path)

A continuación, copiará la división del tren de los datos descargados para que pueda ser consumida por el componente ExampleGen en el siguiente paso. Este componente requiere que sus archivos estén en un directorio sin archivos adicionales (por ejemplo, archivos JSON y TXT).

In [ ]:
# Define the train tfrecord filename
train_filename = 'fashion_mnist-train.tfrecord-00000-of-00001'

# Copy the train tfrecord into the data root folder
!cp {tfds_data_path}/{train_filename} {_data_root}

In [ ]:
!echo {tfds_data_path}/{train_filename} {_data_root}

## Tubería TFX

Una vez completada la configuración, puede proceder a crear la tubería. 

### Inicializar el Contexto Interactivo

Comenzará inicializando el [Contexto Interactivo](https://github.com/tensorflow/tfx/blob/master/tfx/orchestration/experimental/interactive/interactive_context.py) para poder ejecutar los componentes dentro de este entorno Colab. Puede ignorar la advertencia porque sólo utilizará un archivo local SQLite para el almacenamiento de metadatos.

In [ ]:
# Initialize the InteractiveContext
context = InteractiveContext(pipeline_root=_pipeline_root)

### ExampleGen

Comenzará el pipeline ingiriendo el TFRecord que haya apartado. El [ImportExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/ImportExampleGen) consume TFRecords y puede especificar divisiones como se muestra a continuación. Para este ejercicio, usted dividirá el tfrecord de entrenamiento para usar el 80% para el conjunto de entrenamiento, y el 20% restante como conjunto de evaluación/validación.

In [ ]:
# Specify 80/20 split for the train and eval set
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name = 'train', hash_buckets = 8),
        example_gen_pb2.SplitConfig.Split(name = 'eval',  hash_buckets = 2),
    ]))

# Ingest the data through ExampleGen
example_gen = ImportExampleGen(input_base = _data_root, output_config = output)

# Run the component
context.run(example_gen)

In [ ]:
# Print split names and URI
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

### StatisticsGen

A continuación, calculará las estadísticas del conjunto de datos con el componente [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen).

In [ ]:
# Run StatisticsGen
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

context.run(statistics_gen)

### SchemaGen

A continuación, puede inferir el esquema del conjunto de datos con [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen). Esto se utilizará para validar los datos entrantes para asegurar que están formateados correctamente.

In [ ]:
# Run SchemaGen
schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

In [ ]:
# Visualize the results
context.show(schema_gen.outputs['schema'])

### ExampleValidator

Se puede suponer que el conjunto de datos está limpio ya que lo hemos descargado de TFDS. Pero sólo para revisar, vamos a ejecutarlo a través de [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) para detectar si hay anomalías dentro del conjunto de datos.

In [ ]:
# Run ExampleValidator
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

In [ ]:
# Visualize the results. There should be no anomalies.
context.show(example_validator.outputs['anomalies'])

### Transform

Utilicemos ahora el componente [Transform](https://www.tensorflow.org/tfx/guide/transform) para escalar los píxeles de la imagen y convertir los tipos de datos a float. Primero definiremos el módulo de transformación que contiene estas operaciones antes de ejecutar el componente.

In [ ]:
_transform_module_file = 'fmnist_transform.py'

In [ ]:
%%writefile {_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Keys
_LABEL_KEY = 'label'
_IMAGE_KEY = 'image'


def _transformed_name(key):
    return key + '_xf'

def _image_parser(image_str):
    '''converts the images to a float tensor'''
    image = tf.image.decode_image(image_str, channels=1)
    image = tf.reshape(image, (28, 28, 1))
    image = tf.cast(image, tf.float32)
    return image


def _label_parser(label_id):
    '''converts the labels to a float tensor'''
    label = tf.cast(label_id, tf.float32)
    return label


def preprocessing_fn(inputs):
    """Función de retorno de tf.transform para el preprocesamiento de entradas.
    Args:
        inputs: mapa de claves de características a características crudas aún no transformadas.
    Devuelve:
        Mapa de claves de características de cadena a operaciones de características transformadas.
    """
    
    # Convert the raw image and labels to a float array
    with tf.device("/cpu:0"):
        outputs = {
            _transformed_name(_IMAGE_KEY):
                tf.map_fn(
                    _image_parser,
                    tf.squeeze(inputs[_IMAGE_KEY], axis=1),
                    dtype=tf.float32),
            _transformed_name(_LABEL_KEY):
                tf.map_fn(
                    _label_parser,
                    inputs[_LABEL_KEY],
                    dtype=tf.float32)
        }
    
    # scale the pixels from 0 to 1
    outputs[_transformed_name(_IMAGE_KEY)] = tft.scale_to_0_1(outputs[_transformed_name(_IMAGE_KEY)])
    
    return outputs

You will run the component by passing in the examples, schema, and transform module file.

*Note: You can safely ignore the warnings and `udf_utils` related errors.*

In [ ]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Setup the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_transform_module_file))

# Run the component
context.run(transform)

### Tuner

Como su nombre indica, el componente [Tuner](https://www.tensorflow.org/tfx/guide/tuner) ajusta los hiperparámetros de su modelo. Para utilizarlo, tendrá que proporcionar un *archivo de módulo tuner* que contenga una función `tuner_fn()`. En esta función, usted hará en su mayoría los mismos pasos que hizo en el laboratorio anterior no calificado, pero con algunas diferencias clave en el manejo del conjunto de datos. 

El componente Transform guardó anteriormente los ejemplos transformados como TFRecords comprimidos en formato `.gz` y necesitará cargarlo en la memoria. Una vez cargado, necesitará crear lotes de características y etiquetas para que finalmente pueda utilizarlo para la hibridación. Este proceso está modularizado en el `_input_fn()` de abajo. 

Volviendo, la función `tuner_fn()` devolverá un `TunerFnResult` [namedtuple](https://docs.python.org/3/library/collections.html#collections.namedtuple) que contiene su objeto `tuner` y un conjunto de argumentos para pasar al método `tuner.search()`. Verás esto en acción en las siguientes celdas. Cuando revises el archivo del módulo, te recomendamos que veas primero la función `tuner_fn()` antes de ver las otras funciones auxiliares.

In [ ]:
# Declare name of module file
_tuner_module_file = 'tuner.py'

In [ ]:
%%writefile {_tuner_module_file}

# Define imports
from kerastuner.engine import base_tuner
import kerastuner as kt
from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

# Declare namedtuple field names
TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])

# Label key
LABEL_KEY = 'label_xf'

# Callback for the search strategy
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


def _gzip_reader_fn(filenames):
  '''Lconjunto de datos comprimidos oad
  
  Args:
    filenames - nombres de archivos de TFRecords a cargar

  Devuelve:
    TFRecordDataset cargado a partir de los nombres de archivo
  '''

  # Cargar el conjunto de datos. Especifica el tipo de compresión ya que se guarda como `.gz`.
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
  

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
  '''Crear lotes de características y etiquetas a partir de registros TF

  Args:
    file_pattern - Lista de archivos o patrones de rutas de archivos que contienen registros de ejemplo.
    tf_transform_output - Gráfico de salida de la transformación
    num_epochs - Número entero que especifica el número de veces que hay que leer el conjunto de datos. 
            Si es None, se recorre el conjunto de datos para siempre.
    batch_size - Un int que representa el número de registros a combinar en un solo lote.

  Devuelve:
    Un conjunto de datos de elementos dict, (o una tupla de elementos dict y etiqueta). 
    Cada dict asigna claves de características a objetos Tensor o SparseTensor.
  '''

  # Get feature specification based on transform output
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
  
  # Create batches of features and labels
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=LABEL_KEY)
  
  return dataset


def model_builder(hp):
  '''
  Construye el modelo y establece los hiperparámetros a afinar.

  Args:
    hp - Objeto Keras tuner

  Devuelve:
    Modelo con los hiperparámetros a sintonizar
  '''

  # Inicializar la API secuencial y empezar a apilar las capas
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28, 1)))

  # Ajuste el número de unidades en la primera capa densa
  # Elija un valor óptimo entre 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu', name='dense_1'))

  # Add next layers
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(10, activation='softmax'))

  # Ajuste la tasa de aprendizaje para el optimizador
  # Elija un valor óptimo entre 0,01, 0,001 o 0,0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

  return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
  """Construye el sintonizador utilizando la API KerasTuner.
  Argumentos:
    fn_args: contiene los argumentos como pares nombre/valor.

      - working_dir: directorio de trabajo para el ajuste.
      - train_files: Lista de rutas de archivos que contienen datos de entrenamiento tf.Example.
      - eval_files: Lista de rutas de archivos que contienen datos de tf.Example de evaluación.
      - train_steps: número de pasos de entrenamiento.
      - eval_steps: número de pasos de evaluación.
      - schema_path: esquema opcional de los datos de entrada.
      - transform_graph_path: gráfico de transformación opcional producido por TFT.
  
  Devuelve:
    Una namedtuple que contiene lo siguiente:
      - tuner: un BaseTuner que se utilizará para el ajuste.
      - fit_kwargs: Args para pasar a la función run_trial del sintonizador para ajustar el
                    modelo, por ejemplo, el conjunto de datos de entrenamiento y validación. Se requiere
                    depende de la implementación del sintonizador anterior.
  """

  # Define tuner search strategy
  tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory=fn_args.working_dir,
                     project_name='kt_hyperband')

  # Load transform output
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

  # Use _input_fn() to extract input features and labels from the train and val set
  train_set = _input_fn(fn_args.train_files[0], tf_transform_output)
  val_set = _input_fn(fn_args.eval_files[0], tf_transform_output)


  return TunerFnResult(
      tuner=tuner,
      fit_kwargs={ 
          "callbacks":[stop_early],
          'x': train_set,
          'validation_data': val_set,
          'steps_per_epoch': fn_args.train_steps,
          'validation_steps': fn_args.eval_steps
      }
  )

Con el módulo definido, ahora puede configurar el componente Tuner. Puedes ver la descripción de cada argumento [aquí](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/Tuner). 

Fíjate que pasamos un argumento `num_steps` a los argumentos train y eval y esto fue usado en los argumentos `steps_per_epoch` y `validation_steps` en el módulo tuner de arriba. Esto puede ser útil si no se quiere recorrer todo el conjunto de datos al afinar. Por ejemplo, si tienes 10GB de datos de entrenamiento, sería increíblemente lento si lo recorres por completo sólo para una época y un conjunto de hiperparámetros. Puede establecer el número de pasos para que su programa sólo pase por una fracción del conjunto de datos. 

Usted puede calcular el número total de pasos en una época por: `número de ejemplos / tamaño del lote`. Para este ejemplo en particular, tenemos `48000 ejemplos / 32 (tamaño por defecto)` lo que equivale a `1500` pasos por época para el conjunto de entrenamiento (calcular los pasos val de 12000 ejemplos). Dado que has pasado `500` en el `num_steps` de los argumentos del tren, esto significa que algunos ejemplos se saltarán. Esto probablemente resultará en lecturas de menor precisión pero ahorrará tiempo al hacer el hipertuning. Intente modificar este valor más tarde y vea si llega al mismo conjunto de hiperparámetros.

In [ ]:
from tfx.proto import trainer_pb2

# Setup the Tuner component
tuner = Tuner(
    module_file=_tuner_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=100)
    )

In [ ]:
# Run the component. This will take around 10 minutes to run.
# When done, it will summarize the results and show the 10 best trials.
context.run(tuner, enable_cache=False)

### Trainer

Al igual que el componente Tuner, el componente [Trainer](https://www.tensorflow.org/tfx/guide/trainer) también requiere un archivo de módulo para configurar el proceso de entrenamiento. Buscará una función `run_fn()` que defina y entrene el modelo. Los pasos serán similares a los del archivo de módulo del sintonizador:

* Definir el modelo - Puedes obtener los resultados del componente Tuner a través del argumento `fn_args.hyperparameters`. Lo verás pasado a la función `model_builder()` más abajo. Si no has ejecutado `Tuner`, entonces puedes definir explícitamente el número de unidades ocultas y la tasa de aprendizaje.

* Cargar los conjuntos de entrenamiento y validación - Esto lo has hecho en el componente Tuner. Para este módulo, pasarás un valor `num_epochs` (10) para indicar cuántos lotes serán preparados. Puede optar por no hacer esto y pasar un valor `num_steps` como antes.

* Configurar y entrenar el modelo - Esto te resultará muy familiar si ya estás acostumbrado a la [Keras Models Training API](https://keras.io/api/models/model_training_apis/). Puedes pasar callbacks como el [TensorBoard callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) para poder visualizar los resultados más tarde.

* Guarda el modelo - Esto es necesario para que puedas analizar y servir tu modelo. Esto lo harás en partes posteriores del curso y de la especialización.

In [ ]:
# Declare trainer module file
_trainer_module_file = 'trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

# Define the label key
LABEL_KEY = 'label_xf'

def _gzip_reader_fn(filenames):
  '''Cargar un conjunto de datos comprimidos
  
  Args:
    filenames - nombres de archivos de TFRecords a cargar

  Devuelve:
    TFRecordDataset cargado a partir de los nombres de archivo
  '''

  # Cargar el conjunto de datos. Especifica el tipo de compresión ya que se guarda como `.gz`.
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
  

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
  '''Crear lotes de características y etiquetas a partir de registros TF

  Args:
    file_pattern - Lista de archivos o patrones de rutas de archivos que contienen registros de ejemplo.
    tf_transform_output - Gráfico de salida de la transformación
    num_epochs - Número entero que especifica el número de veces que hay que leer el conjunto de datos. 
            Si es None, se recorre el conjunto de datos para siempre.
    batch_size - Un int que representa el número de registros a combinar en un solo lote.

  Devuelve:
    Un conjunto de datos de elementos dict, (o una tupla de elementos dict y etiqueta). 
    Cada dict asigna claves de características a objetos Tensor o SparseTensor.
  '''
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
  
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=LABEL_KEY)
  
  return dataset


def model_builder(hp):
  '''
  Construye el modelo y establece los hiperparámetros a afinar.

  Args:
    hp - objeto Keras tuner

  Devuelve:
    Modelo con los hiperparámetros a sintonizar
  '''

  # Inicializar la API secuencial y empezar a apilar las capas
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28, 1)))

  # Get the number of units from the Tuner results
  hp_units = hp.get('units')
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))

  # Add next layers
  model.add(keras.layers.Dropout(0.2))
  model.add(keras.layers.Dense(10, activation='softmax'))

  # Get the learning rate from the Tuner results
  hp_learning_rate = hp.get('learning_rate')

  # Setup model for training
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

  # Print the model summary
  model.summary()
  
  return model


def run_fn(fn_args: FnArgs) -> None:
  """Define y entrena el modelo.
  Argumentos:
    fn_args: Contiene los argumentos como pares nombre/valor. Consulte aquí los atributos completos: 
    https://www.tensorflow.org/tfx/api_docs/python/tfx/components/trainer/fn_args_utils/FnArgs#attributes
  """

  # Callback for TensorBoard
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')
  
  # Load transform output
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
  
  # Create batches of data good for 10 epochs
  train_set = _input_fn(fn_args.train_files[0], tf_transform_output, 10)
  val_set = _input_fn(fn_args.eval_files[0], tf_transform_output, 10)

  # Load best hyperparameters
  hp = fn_args.hyperparameters.get('values')

  # Build the model
  model = model_builder(hp)

  # Train the model
  model.fit(
      x=train_set,
      validation_data=val_set,
      callbacks=[tensorboard_callback]
      )
  
  # Save the model
  model.save(fn_args.serving_model_dir, save_format='tf')

Puedes pasar la salida del componente `Tuner` al `Trainer` rellenando el argumento `hyperparameters` con la salida del `Tuner`. Esto se indica con el argumento `tuner.outputs['best_hyperparameters']` más abajo. Puedes ver la definición de los otros argumentos [aquí](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/Trainer).

In [ ]:
# Setup the Trainer component
trainer = Trainer(
    module_file     =_trainer_module_file,
    examples        = transform.outputs['transformed_examples'],
    hyperparameters = tuner.outputs['best_hyperparameters'],
    transform_graph = transform.outputs['transform_graph'],
    schema          = schema_gen.outputs['schema'],
    train_args      = trainer_pb2.TrainArgs(splits=['train']),
    eval_args       = trainer_pb2.EvalArgs(splits=['eval']))

Tenga en cuenta que al reentrenar su modelo, no siempre tiene que reajustar sus hiperparámetros. Una vez que tengas un conjunto que creas que tiene un buen rendimiento, puedes simplemente importarlo con el ImporterNode como se muestra en los [docs oficiales](https://www.tensorflow.org/tfx/guide/tuner):

```
hparams_importer = ImporterNode(
    instance_name='import_hparams',
    # This can be Tuner's output file or manually edited file. The file contains
    # text format of hyperparameters (kerastuner.HyperParameters.get_config())
    source_uri='path/to/best_hyperparameters.txt',
    artifact_type=HyperParameters)

trainer = Trainer(
    ...
    # An alternative is directly use the tuned hyperparameters in Trainer's user
    # module code and set hyperparameters to None here.
    hyperparameters = hparams_importer.outputs['result'])
```

In [ ]:
# Run the component
context.run(trainer, enable_cache=False)

Su modelo debería estar ahora guardado en el directorio de su pipeline y puede navegar por él como se muestra a continuación. El archivo se guarda como `saved_model.pb`.

In [ ]:
# Get artifact uri of trainer model output
model_artifact_dir = trainer.outputs['model'].get()[0].uri

# List subdirectories artifact uri
print(f'contents of model artifact directory:{os.listdir(model_artifact_dir)}')

# Define the model directory
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')

# List contents of model directory
print(f'contents of model directory: {os.listdir(model_dir)}')

También puedes visualizar los resultados del entrenamiento cargando los registros guardados por el callback del Tensorboard.

In [ ]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

***¡Felicidades! Ahora ha creado un pipeline de ML que incluye el ajuste de hiperparámetros y el entrenamiento del modelo. Sabrá más sobre los siguientes componentes en futuras lecciones, pero en la siguiente sección, primero aprenderá sobre un marco de trabajo para construir automáticamente tuberías de ML: AutoML. Disfruta del resto del curso.***